In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
juvenile_df = pd.read_csv('../Data/juvenile.csv', dtype = {5: str, 7: str}, header = None)

## Rename Columns

In [4]:
juvenile_df.columns = ['UsageClass', 'CheckoutType', 'MaterialType', 'CheckoutYear', 'CheckoutMonth', 'Checkouts', 'Title', 'ISBN', 'Creator', 'Subjects', 'Publisher', 'PublicationYear']

## Adjust capitalization and data types

In [5]:
juvenile_df['MaterialType'] = juvenile_df.MaterialType.str.title()

In [6]:
juvenile_df[['Title', 'TitleNotes']] = juvenile_df.Title.str.split(' / ', expand = True, n = 1)

In [7]:
juvenile_df['Title'] = [' '.join([w.capitalize() for w in x]) for x in juvenile_df.Title.str.split(' ')]

## To Do: capitalize first character after a (, use title case unless there's an apostrophe, keep all-caps words in all-caps?

In [8]:
juvenile_df['Checkouts'] = juvenile_df.Checkouts.str.replace(',', '').astype(int)

In [34]:
juvenile_df['CheckoutDate'] = juvenile_df['CheckoutYear'].astype(str) + '-' + juvenile_df['CheckoutMonth'].astype(str) + '-01'

## Extract Creator names in `Firstname Lastname` format

In [9]:
##To Do - look into juvenile_df.loc[juvenile_df.Creator.isna()]

In [10]:
juvenile_df['Creator'] = juvenile_df.Creator.str.strip(',')

In [11]:
lastnames = [re.search(r'^(.+?),', c).group(1) if str(c).find(',') != -1 else c for c in juvenile_df.Creator]

In [12]:
firstnames = [re.search(r'^(.+?),([A-Za-zÀ-ÿ\-\'\s.]+)', c).group(2) if str(c).find(',') != -1 else '' for c in juvenile_df.Creator]

In [13]:
fullnames = []
i = 0

for name in lastnames:
    fullname = (str(firstnames[i]).strip() + ' ' + str(name).strip()).strip()
    fullnames.append(fullname)
    i += 1

In [14]:
juvenile_df['CreatorName'] = fullnames

## Extract dates 

#### Replace placeholders (`-`) with 0s, followed by `?` in incomplete years

In [15]:
incomplete_dates = juvenile_df.loc[(~juvenile_df.PublicationYear.isna()) & (juvenile_df.PublicationYear.str.contains('-')), 'PublicationYear'].str.extract(r'[\b\D](\d{2,3}\-{1,2})')

In [16]:
incomplete_dates = incomplete_dates.loc[~incomplete_dates[0].isna()]

In [17]:
incomplete_dates['new'] = incomplete_dates[0].str.replace('-', '0') + '?'

In [18]:
for i, r in incomplete_dates.iterrows():
    juvenile_df.loc[i, 'PublicationYear'] = juvenile_df.loc[i, 'PublicationYear'].replace(r[0], r['new']).replace('??', '?')

### Extract different types of PublicationYear values  

Examples of formatting varieties (from dataset FAQ): 
* `2005` — publication date
* `c. 2005` – copyright symbol
* `[2005]` – Printing date
* `p. 2005` – phonogram copyright symbol
* `2004, c. 2005` – publication and copyright date
* `2005-2007` – intervening years
* `[2005?]` – Approximate date

#### Copyright Dates
* Variants include: c. YYYY, C. YYYY, ©YYYY, © YYYY, cop. YYYY, c&p YYYY and similar
* Multiple dates may be listed, potentially in any order

In [19]:
juvenile_df['CopyrightList'] = juvenile_df.PublicationYear.str.findall(r'[©cC](?:op)?(?:&p)?\.?\s?(\d{4})')

In [20]:
juvenile_df['CopyrightList'] = juvenile_df.CopyrightList.fillna('').apply(list)

In [21]:
juvenile_df['CopyrightDate'] = [int(max(x)) if len(x) > 0 else np.nan for x in juvenile_df['CopyrightList']]

#### Phonogram Copyright Dates
* Variants include: p. YYYY, pYYYY, P.YYYY, c&p YYYY and similar
* Multiple dates could be listed, potentially in any order

In [22]:
juvenile_df['PCopyrightList'] = juvenile_df.PublicationYear.str.findall(r'\b(?:c&)?[pP]\.?\s?(\d{4})')

In [23]:
juvenile_df['PCopyrightList'] = juvenile_df.PCopyrightList.fillna('').apply(list)

In [24]:
juvenile_df['PCopyrightDate'] = [int(max(x)) if len(x) > 0 else np.nan for x in juvenile_df['PCopyrightList']]

#### Printing Dates
* Printing dates will always be enclosed in square brackets
* I do NOT want approximate dates (which are followed by question marks) or copyright dates

In [25]:
juvenile_df['PrintingDate'] = juvenile_df.PublicationYear.str.extract(r'\[.*(?<![©cCpP])[\.\s&opP]{0,5}(\d{4})(?!\?).*\]')

#### Approximate Dates
* Enclosed in brackets and followed by a question mark
* There may be a range of dates followed by a question mark `YYYY - YYYY?`  in which case I want to return the earliest (first) year of the range

In [26]:
juvenile_df['ApproxDate'] = juvenile_df.PublicationYear.str.extract(r'\[.*?(\d{4}).*\?\]')

#### Publication Dates
* Publication dates are noted by a lack of previous signifiers - they are NOT in brackets and NOT preceded by a copyright signifier

In [27]:
juvenile_df['PubDate'] = juvenile_df.PublicationYear.str.extract(r'(?<![©cCpP\[])[\.\s&opP]{0,5}(\d{4})(?!\s?[\?\]-])')

#### Other Dates
* Pull out any years that manage to avoid meeting any previous criteria

In [28]:
juvenile_df['OtherDateList'] = np.nan

In [29]:
juvenile_df['OtherDateList'] = np.where(juvenile_df.PubDate.isna() & juvenile_df.PrintingDate.isna() & juvenile_df.CopyrightDate.isna() & juvenile_df.PCopyrightDate.isna() & juvenile_df.ApproxDate.isna(), 
                                     juvenile_df.PublicationYear.str.findall(r'(\d{4})'),
                                     np.nan)

In [30]:
juvenile_df['OtherDateList'] = juvenile_df.OtherDateList.fillna('').apply(list)

In [31]:
juvenile_df['OtherDate'] = [int(max(x)) if len(x) > 0 else np.nan for x in juvenile_df['OtherDateList']]

### Consolidate to 'CreatedDate' for use in analysis

In [32]:
juvenile_df['CreatedDate'] = juvenile_df[['PubDate', 'PrintingDate', 'CopyrightDate', 'PCopyrightDate', 'ApproxDate', 'OtherDate']].bfill(axis = 1)['PubDate']

C:\Users\5pira\AppData\Local\Temp\ipykernel_17568\3170989505.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  juvenile_df['CreatedDate'] = juvenile_df[['PubDate', 'PrintingDate', 'CopyrightDate', 'PCopyrightDate', 'ApproxDate', 'OtherDate']].bfill(axis = 1)['PubDate']


## Export cleaned data

In [35]:
juvenile_df.to_csv('../Data/juvenile_clean.csv')

## Transform for analysis in Power BI

In [236]:
juvenile_df.head()

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,...,CopyrightDate,PCopyrightDate,PrintingDate,ApproxDate,PubDate,CopyrightList,PCopyrightList,OtherDate,OtherDateList,CreatedDate
0,Physical,Horizon,Book,2023,10,1,Don't Hate The Player,"1547605022, 9781547605026, 9781547605033","Nedd, Alexis","eSports Contests Fiction, High schools Fiction...",...,NaN,NaN,NaN,NaN,2021,[],[],NaN,[],2021
1,Physical,Horizon,Book,2023,10,1,Thanksgiving Graces,"0824956346, 9780824956349","Moulton, Mark Kimball","Stories in rhyme Juvenile fiction, Thanksgivin...",...,NaN,NaN,2011,NaN,NaN,[],[],NaN,[],2011
2,Physical,Horizon,Book,2023,10,1,The Elephant Thief,"1338188437, 9781338188431","Kerr, Jane (Children's literature author)","Pickpockets Juvenile fiction, Zoo keepers Juve...",...,NaN,NaN,NaN,NaN,2018,[],[],NaN,[],2018
3,Physical,Horizon,Book,2023,10,1,Elsie's Bird,"0399252924, 9780399252921","Yolen, Jane","Prairies Juvenile fiction, Frontier and pionee...",...,NaN,NaN,2010,NaN,NaN,[],[],NaN,[],2010
4,Physical,Horizon,Book,2023,10,6,The Ogre In The Hall,"1662640528, 9781662640520","Sorin, Céline","Monsters Juvenile fiction, Fear of the dark Ju...",...,NaN,NaN,2023,NaN,NaN,[],[],NaN,[],2023


In [244]:
book_df = juvenile_df[['Title', 'TitleNotes', 'ISBN', 'Creator', 'PublicationYear', 'CreatedDate', 'PubDate', 'PrintingDate', 'CopyrightDate', 'PCopyrightDate', 'ApproxDate', 'OtherDate']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index':'bookID'})

In [293]:
book_df.to_csv('../Data/books.csv')

In [247]:
checkouts_df = pd.merge(juvenile_df, book_df, how = 'left', on = ['Title', 'TitleNotes', 'ISBN', 'Creator', 'PublicationYear', 'CreatedDate', 'PubDate', 'PrintingDate', 'CopyrightDate', 'PCopyrightDate', 'ApproxDate', 'OtherDate'])[['CheckoutYear', 'CheckoutMonth', 'UsageClass', 'CheckoutType', 'MaterialType', 'bookID']]

In [294]:
checkouts_df.to_csv('../Data/checkouts.csv')

In [264]:
subject_df = pd.merge(juvenile_df, book_df, how = 'left', on = ['Title', 'TitleNotes', 'ISBN', 'Creator', 'PublicationYear', 'CreatedDate', 'PubDate', 'PrintingDate', 'CopyrightDate', 'PCopyrightDate', 'ApproxDate', 'OtherDate'])[['bookID', 'Subjects']].drop_duplicates().reset_index(drop = True)

In [272]:
subject_df = subject_df.rename(columns = {'Subjects': 'AllSubjects'})

In [288]:
all_subjects = pd.DataFrame(subject_df.AllSubjects.str.split(r',').explode().str.strip().str.strip(',')).rename(columns = {'AllSubjects': 'Subjects'})

In [291]:
subject_df = pd.merge(all_subjects, subject_df, how = 'left', left_index = True, right_index = True)[['bookID', 'Subjects']].reset_index(drop = True)

In [295]:
subject_df.to_csv('../Data/subjects.csv')